# Introduction to [AI Search](https://learn.microsoft.com/en-us/azure/search/search-what-is-an-index)
In Azure AI Search, a search index is your searchable content, available to the search engine for indexing, full text search, vector search, hybrid search, and filtered queries.

An index is defined by a schema and saved to the search service, with data import following as a second step. 

This content exists within your search service, apart from your primary data stores, which is necessary for the millisecond response times expected in modern search applications. Except for indexer-driven indexing scenarios, the search service never connects to or queries your source data.

## 1. Content (documents and schema)

In Azure AI Search, indexes contain search documents. Conceptually, a document is a single unit of searchable data in your index.

Mapping these concepts to more familiar database equivalents: a search index equates to a table, and documents are roughly equivalent to rows in a table.

The structure of a document is determined by the index schema, as illustrated in the following example. The "fields" collection is typically the largest part of an index, where each field is named, assigned a [data type](https://learn.microsoft.com/en-us/rest/api/searchservice/Supported-data-types), and attributed with allowable behaviors that determine how it's used.



```
{
  "name": "name_of_index, unique across the service",
  "fields": [
    {
      "name": "name_of_field",
      "type": "Edm.String | Collection(Edm.String) | Collection(Edm.Single) | Edm.Int32 | Edm.Int64 | Edm.Double | Edm.Boolean | Edm.DateTimeOffset | Edm.GeographyPoint",
      "searchable": true (default where applicable) | false (only Edm.String and Collection(Edm.String) fields can be searchable),
      "filterable": true (default) | false,
      "sortable": true (default where applicable) | false (Collection(Edm.String) fields cannot be sortable),
      "facetable": true (default where applicable) | false (Edm.GeographyPoint fields cannot be facetable),
      "key": true | false (default, only Edm.String fields can be keys),
      "retrievable": true (default) | false,
      "analyzer": "name_of_analyzer_for_search_and_indexing", (only if 'searchAnalyzer' and 'indexAnalyzer' are not set)
      "searchAnalyzer": "name_of_search_analyzer", (only if 'indexAnalyzer' is set and 'analyzer' is not set)
      "indexAnalyzer": "name_of_indexing_analyzer", (only if 'searchAnalyzer' is set and 'analyzer' is not set)
      "normalizer":  "name_of_normalizer", (applies to fields that are filterable)
      "synonymMaps": "name_of_synonym_map", (optional, only one synonym map per field is currently supported)
      "dimensions": "number of dimensions used by an emedding models", (applies to vector fields only, of type Collection(Edm.Single))
      "vectorSearchProfile": "name_of_vector_profile" (indexes can have many configurations, a field can use just one)
    }
  ],
  "suggesters": [ ],
  "scoringProfiles": [ ],
  "analyzers":(optional)[ ... ],
  "charFilters":(optional)[ ... ],
  "tokenizers":(optional)[ ... ],
  "tokenFilters":(optional)[ ... ],
  "defaultScoringProfile": (optional) "...",
  "corsOptions": (optional) { },
  "encryptionKey":(optional){ },
  "semantic":(optional){ },
  "vectorSearch":(optional){ }
}
```

- suggesters support type-ahead queries like autocomplete.
- scoringProfiles are used for relevance tuning.
- analyzers are used to process strings into tokens according to linguistic rules or other characteristics supported by the analyzer.
- corsOptions, or Cross-origin remote scripting (CORS), is used for apps that issues requests from different domains.
- encryptionKey configures double-encryption of sensitive content in the index.
- semantic configures semantic reranking in full text and hybrid search.
- vectorSearch configures vector fields and queries.

### Field definitions

A search document is defined by the "fields" collection in the body of Create Index request. You need fields for document identification (keys), storing searchable text, and fields for supporting filters, facets, and sorting

You might also need fields for data that a user never sees. For example, you might want fields for profit margins or marketing promotions that you can use in a scoring profile to boost a search score.



### Field Attributes

Field attributes determine how a field is used, such as whether it's used in full text search, faceted navigation, sort operations, and so forth.

String fields are often marked as "searchable" and "retrievable". Fields used to narrow search results include "sortable", "filterable", and "facetable".

Certainly! Here's the information formatted in markdown:

| **Attribute** | **Description**                                                                                                                                                                                                                   |
|---------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| "searchable"  | Full-text or vector searchable. Text fields are subject to lexical analysis such as word-breaking during indexing. If you set a searchable field to a value like "sunny day", internally it's split into the individual tokens "sunny" and "day". For details, see How full text search works. |
| "filterable"  | Referenced in filter queries. Filterable fields of type Edm.String or Collection(Edm.String) don't undergo word-breaking, so comparisons are for exact matches only. For example, if you set such a field f to "sunny day", filter=f eq 'sunny' finds no matches, but filter=f eq 'sunny day' will. |
| "sortable"    | By default, the system sorts results by score, but you can configure sorting based on fields in the documents. Fields of type Collection(Edm.String) can't be "sortable".                                                              |
| "facetable"   | Typically used in a presentation of search results that includes a hit count by category (for example, hotels in a specific city). This option can't be used with fields of type Edm.GeographyPoint. Fields of type Edm.String that are filterable, "sortable", or "facetable" can be at most 32 kilobytes in length. For details, see Create Index (REST API). |
| "key"         | Unique identifier for documents within the index. Exactly one field must be chosen as the key field, and it must be of type Edm.String.                                                                                             |
| "retrievable" | Determines whether the field can be returned in a search result. This is useful when you want to use a field (such as profit margin) as a filter, sorting, or scoring mechanism but don't want the field to be visible to the end user. This attribute must be true for key fields. |
